### 1. General info of dataset GSE227122

This is the Jupyter Notebook for dataset GSE227122. Its dataset includes barcodes/features/matrix files for each sample.

Thus, we need to simply incorparate these barcodes/features/matrix files and generate an AnnData object for each sample. 
In total, there are 16 T-cell acute lymphoblastic leukemia (T-ALL) samples.

<span style="color:green">**[Dx]**</span> patients at diagnosis

<span style="color:green">**[EOI]**</span> patients after therapy

<span style="color:green">**[Rel]**</span> patients at relapse

In [7]:
# Environment setup
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as anndata
import scipy

### 2. AnnData object of each sample

<span style="color:red">**IMPORTANT:**</span> rename files to get rid of prefixes 

1. `barcodes.tsv.gz`: cell barcodes, which go into `.obs`
2. `features.tsv.gz`: gene names, `.var`
3. `matrix.mtx.gz`: the expression matrix, `.X`

In [ ]:
from pathlib import Path

# Specify directory paths
data_directory = Path('/scratch/user/s4543064/xiaohan-john-project/data/GSE227122')
write_directory = Path('/scratch/user/s4543064/xiaohan-john-project/write/GSE227122')

# Loop through all files in the directory
for sample_directory in data_directory.iterdir():
    sample_name = sample_directory.stem
    sample_h5ad = sample_name + '_uni.h5ad'

    sample = sc.read_10x_mtx(
    sample_directory,
    var_names='gene_symbols',  
    cache=False
    )

    # Create an observation metric info to store related features
    obs_metrics = pd.DataFrame(index=sample.obs_names) ## Get the identifiers
    obs_metrics['cancer_type'] = 'T-cell_acute_lymphoblastic_leukemia'
    obs_metrics['dataset'] = 'GSE227122'
    obs_metrics['tissue'] = 'bone_marrow'
    obs_metrics['uni_barcode'] = obs_metrics['dataset'] + '_' + obs_metrics.index.astype(str)
    
    sample.obs = obs_metrics
    sample.obs.set_index("uni_barcode", drop=False, inplace=True)
    print(sample)

    # save the anndata object
    output_path = write_directory / sample_h5ad
    sample.write_h5ad(output_path, compression="gzip")

### 3. Confirmation of created AnnData objects

In [2]:
from pathlib import Path

# Specify directory paths
write_directory = Path('/scratch/user/s4543064/xiaohan-john-project/write/GSE227122')

# Loop through all files in the directory
for file in write_directory.iterdir():
    sample = anndata.read_h5ad(file)
    print(sample)

AnnData object with n_obs × n_vars = 3522 × 33538
    obs: 'cancer_type', 'dataset', 'tissue', 'uni_barcode'
    var: 'gene_ids', 'feature_types'
AnnData object with n_obs × n_vars = 2894 × 33538
    obs: 'cancer_type', 'dataset', 'tissue', 'uni_barcode'
    var: 'gene_ids', 'feature_types'
AnnData object with n_obs × n_vars = 3530 × 33538
    obs: 'cancer_type', 'dataset', 'tissue', 'uni_barcode'
    var: 'gene_ids', 'feature_types'
AnnData object with n_obs × n_vars = 1073 × 33538
    obs: 'cancer_type', 'dataset', 'tissue', 'uni_barcode'
    var: 'gene_ids', 'feature_types'
AnnData object with n_obs × n_vars = 949 × 33538
    obs: 'cancer_type', 'dataset', 'tissue', 'uni_barcode'
    var: 'gene_ids', 'feature_types'
AnnData object with n_obs × n_vars = 1845 × 33538
    obs: 'cancer_type', 'dataset', 'tissue', 'uni_barcode'
    var: 'gene_ids', 'feature_types'
AnnData object with n_obs × n_vars = 4034 × 33538
    obs: 'cancer_type', 'dataset', 'tissue', 'uni_barcode'
    var: 'gene_i

### 4. Convert AnnData objects to SingleCellExperiment objects

<span style="color:yellow">**Attempt 1:**</span> usage from Python

In [5]:
import anndata2ri
from rpy2.robjects import r
anndata2ri.activate()

adata = r('as(sample, "SingleCellExperiment")')
adata

ModuleNotFoundError: No module named 'anndata2ri'

In [4]:
!jupyter kernelspec list

Available kernels:
  python3    /home/s4543064/miniforge3/envs/summer_env/share/jupyter/kernels/python3


In [6]:
import sys
sys.executable


'/home/s4543064/miniforge3/envs/summer_env/bin/python'